In [22]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
import numpy as np
import sqlite3

Syftet med den här är att mäta avståndet mellan huvudgruppernas medelpunkter.

In [23]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs limit 4000')
for row in rows:
        result.append(row)

db_contents = np.array(result)
print (db_contents)

conn.close()

[['Talg nöt' '1' 656.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86 None]
 ['Späck gris' '2' 763.0 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ['Ister gris' '3' 884.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ..., 
 ['Havredryck oberikad' '5964' 40.6 ...,
  'Vegetabiliska produkter och mjölkersättning' 46 None]
 ['Kryddblandning taco' '5973' 372.6 ..., 'Kryddor' 112 None]
 ['Tortilla wrap' '5974' 313.7 ..., 'Mjukt bröd' 12 'train']]


In [24]:
columns = np.array([ 2,  4,  5,  6])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]

In [25]:
dataset = db_contents[:,columns].astype(float)
#means = np.mean(dataset, axis=0)

label_types = np.array(list(set(db_contents.T[60]))).reshape(-1, 1) 

#Skapa 
label_types=np.hstack((label_types,np.empty([len(label_types),len(columns)])))

In [62]:
# Fyll matrisen label_types med medelvärdena i alla kolumner
for row in label_types:
    a = np.where(db_contents[:,60]==row[0]) # a blir index för alla livsmedel som tillhör den aktuella huvudgruppen
    x=np.mean(dataset[a], axis=0) # blir medelvärdena för alla kolumner för den aktuella huvudgruppen
    row[1:]=x # Sätt medelvärdena på plats i den stora matrisen som är label_types/huvudgrupper
    print(row)

['' '216.54285714285717' '11.528571428571428' '9.6' '14.385714285714286']
['Salt' '32.075' '5.725' '0.125' '1.9']
['Vin' '92.76000000000002' '7.12' '0.0' '0.01']
['Kakaoprodukter' '398.1' '49.15' '12.2' '14.28']
['Frukt färsk fryst' '58.36904761904763' '11.59928571428571'
 '0.36595238095238103' '0.7402380952380951']
['Dessertost' '329.0625' '1.1375' '27.34375' '20.338749999999997']
['Godis ej choklad' '377.37272727272733' '85.44545454545455'
 '1.9363636363636365' '3.1681818181818175']
['Tuggummi' '365.3' '89.4' '0.0' '0.5']
['Sockerfritt godis' '401.8' '98.9' '0.0' '0.0']
['Hård matfettsblandning' '502.23076923076934' '1.669230769230769'
 '55.82051282051282' '0.5002564102564103']
['Jäst bakpulver' '214.5' '34.86666666666667' '0.6666666666666666'
 '16.400000000000002']
['Mjölkdryck chokladdryck milkshake smothie m yoghurt' '74.94000000000001'
 '10.41' '2.112' '3.3879999999999995']
['Te' '78.3' '15.7' '0.1' '2.925']
['Välling' '214.5833333333333' '28.617500000000003' '6.406666666666666'


In [34]:
label_types[1:5,1:]

array([['32.075', '5.725', '0.125', '1.9'],
       ['92.76000000000002', '7.12', '0.0', '0.01'],
       ['398.1', '49.15', '12.2', '14.28'],
       ['58.36904761904763', '11.59928571428571', '0.36595238095238103',
        '0.7402380952380951']],
      dtype='<U73')

In [35]:
pairwise_distances(label_types[1:5,1:], Y=None, metric='euclidean', n_jobs=1)

array([[   0.        ,   60.73057694,  368.99742719,   26.96826429],
       [  60.73057694,    0.        ,  308.7903972 ,   34.69104748],
       [ 368.99742719,  308.7903972 ,    0.        ,  342.27261939],
       [  26.96826429,   34.69104748,  342.27261939,    0.        ]])

In [38]:
cos_sim = cosine_similarity(label_types[:,1:], Y=None)

In [39]:
cos_sim.shape

(119, 119)

In [44]:
cos_sim[1]

array([ 0.99158365,  1.        ,  0.99331355,  0.99794853,  0.99875767,
        0.98202978,  0.99768878,  0.99636849,  0.99620688,  0.97780484,
        0.99973453,  0.99887717,  0.99953717,  0.99858207,  0.97719208,
        0.99458754,  0.98576591,  0.99460265,  0.98573167,  0.99758861,
        0.99579994,  0.99559678,  0.99766775,  0.99387594,  0.97704976,
        0.98179828,  0.99137614,  0.99135845,  0.99589798,  0.99909171,
        0.98458079,  0.9982447 ,  0.9772922 ,  0.99630561,  0.99072236,
        0.99705881,  0.98193584,  0.99431477,  0.98175452,  0.98207391,
        0.98287088,  0.97535953,  0.98550266,  0.98852824,  0.99930098,
        0.99878028,  0.98282938,  0.98556754,  0.99591759,  0.99490781,
        0.9995786 ,  0.99717899,  0.99951551,  0.99620802,  0.99633574,
        0.99910426,  0.98488998,  0.99626354,  0.98777149,  0.99717503,
        0.98181428,  0.98847533,  0.99581391,  0.99408704,  0.99636871,
        0.96976396,  0.98857345,  0.99964382,  0.99147681,  0.99

In [45]:
np.argsort (cos_sim[1])

array([101,  65, 115,  41,  70,  24,  14,  32,   9,  92, 116,  38,  89,
        25,  60,  36,   5,  86,  39,  46,  40,  91,  30,  98,  56,  42,
        96,  47,  18,  16,  93, 106,  58,  61,  43,  66,  74,  34,  27,
        26,  68,   0,  84,   2, 105,  23,  63,  37,  15,  17,  49,  75,
       110, 114,  21,  20,  62,  28,  94,  48,  87,   8,  53, 104,  76,
       100,  57,  33,  54,   7,  64, 111,  73,  82,  83,  35,  59,  51,
       117, 102,  19,  22,   6,   3, 103,  95,  85,  31,  71, 107,  77,
        90,  13,   4,  45, 108,  11,  80,  88, 112,  29,  55,  81,  72,
       118,  44,  97,  78,  69,  99,  52,  12,  50, 109, 113,  67,  10,
        79,   1])

In [53]:
similar = label_types[np.argsort (cos_sim)][::-1] #[::-1] vänder på listan https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order

In [60]:
similar[4]

array([['Övrigt', '0.0', '0.0', '0.0', '0.0'],
       ['Olja', '876.3533333333331', '0.09000000000000001',
        '99.00933333333334', '0.10733333333333334'],
       ['Flytande matfettsblandning', '713.3714285714285',
        '0.21428571428571427', '80.57142857142857', '0.14285714285714285'],
       ['Smör', '727.9333333333334', '0.48666666666666664',
        '81.86666666666667', '0.5'],
       ['Övrigt fett (ister, talg, kokosfett)', '796.9749999999999', '0.0',
        '89.0', '2.45'],
       ['Hård matfettsblandning', '502.23076923076934',
        '1.669230769230769', '55.82051282051282', '0.5002564102564103'],
       ['Sockerfritt godis', '401.8', '98.9', '0.0', '0.0'],
       ['Sötningsmedel', '406.3', '100.0', '0.0', '0.0'],
       ['Sportdrycker energidrycker', '34.15', '8.4', '0.0', '0.0'],
       ['Socker sirap honung', '365.2', '89.83333333333333', '0.0',
        '0.049999999999999996'],
       ['Saft läsk cider u alkohol', '75.8736842105263',
        '18.610526315789468', '0